<a href="https://colab.research.google.com/github/mugalan/vibration-analysis/blob/main/assignments/Basic_Vibration_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import math
from numpy import linalg
import sympy

from sympy import symbols
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting(pretty_print=True)

import plotly.graph_objects as go

Meeswin engineering are an engineering company that have been commissioned to design an auto-closing gate for use
in rural settings. Because it is to be used in remote locations, it cannot use any electronics. The engineers have decided
it will therefore be a swinging gate mounted to a vertical post about which it rotates, with a torsional spring that cannot be
adjusted which has a torsional spring constant of $k = 130\,N.m$. They will control the torsional damping with a drum brake
that simply uses friction to slow the gate's rotation. The amount of friction can be controlled by tightening or loosening a
bolt which presses a brake shoe against the drum. Both the torsional spring and the brake can be considered linear. The
gate weighs $75\,kg$, with its centre of mass concentrated at its centre point which is $1.5\,m$ from the vertical post. The initial
settings on the gate brake produce a rotational damping coefficient of $c= 10\,N.m/s$. Your tasks are

* a. If the gate is opened to an angle of $\theta = 30^o$, then released from rest, find the expression for the time history of the
motion of the gate.

* b. Provide an explanation of which vibration phenomenon could be exploited to minimise the time that the gate remains
open and how this could be implemented given the gate design.

* c. For the provided gate, calculate new values of any coefficients that would lead to the minimum open time, and calculate
how long it will take the gate to close from an initial position of $\theta = 30^o$ (the gate can be considered closed when its
position is $\theta= 1^o$).

* d. Produce a pdf file called q1.pdf, containing your working and final expression for the motion of the gate, explanation
of a phenomenon to exploit to minimise open time, your calculations and new values of coefficients to minimise open
time, and your calculations of the minimum time the gate will remain open. Upload this file as part of your submission.

## System Model and Initial Condition Response


Assumin that the horizontal gate position is the equilibrium position of the gate we have

Let $\omega^2_n=\frac{K}{I}$ and $2 \zeta \omega_n=\frac{C}{I}$. Then the above equations take the form
\begin{align}
\ddot{\theta}(t) + 2 \zeta \omega_n \dot{\theta}(t) + \omega^2_n \theta(t)=0.
\end{align}

Taking Laplace transform
of both sides of and using the linearity property of the Laplace transform we have
\begin{align}
\mathfrak{L}\left\{\ddot{\theta}(t) + 2 \zeta \omega_n \dot{\theta}(t) + \omega^2_n \theta(t)\right\}&=\frac{1}{M}\mathfrak{L}\left\{f(t)\right\},\\
(s^2+2 \zeta \omega_n s + \omega_n^2) \Theta(s) - \dot{\theta}(0)- ( s + 2\zeta \omega_n)\theta(0)  &= 0.
\end{align}
This yields
\begin{align}
\Theta(s)=\frac{1}{s^2+2 \zeta \omega_n s + \omega_n^2} \dot{\theta}(0)+\frac{ s + 2\zeta \omega_n}{s^2+2 \zeta \omega_n s + \omega_n^2}\theta(0)
\end{align}
Since the Laplace is a one-to-one and onto operator its inverse exists and thus the the solution can be uniquely determined to be $x(t)=\mathfrak{L}^{-1}\{X(s)\}$. Using the linearity property of the inverse we thus have


\begin{align}
\theta(t)&=\dot{\theta}(0)\mathfrak{L}^{-1}\left\{\frac{1}{s^2+2 \zeta \omega_n s + \omega_n^2} \right\}+\theta(0)\mathfrak{L}^{-1}\left\{\frac{ s + 2\zeta \omega_n}{s^2+2 \zeta \omega_n s + \omega_n^2}\right\}.
\end{align}



Assuming underdaped conditions $0\leq\zeta \leq 1$ it is a straight forward exercise in partial fraction expansion to show that the above expressions reduce to

\begin{align}
\theta_{IC} (t) &=e^{-\zeta \omega_nt}\left(\frac{\dot{\theta}(0)}{\omega_d}\cos{(\omega_dt-\pi/2)}+\frac{\theta(0)}{\sqrt{1-\zeta^2}}\sin{(\omega_dt+\phi_{IC})}\right)
\end{align}
where $\omega_d=\omega_n\sqrt{1-\zeta^2}$, $\phi_{IC}=\arcsin\left({\sqrt{1-\zeta^2}}\right)$

In [ ]:
M=75; L=1.5; K=130; C=10; wn=np.sqrt(K/M);
zeta=2*C/wn

In [ ]:
wd=wn*np.sqrt(1-zeta**2); phi_IC=np.arcsin(1-zeta**2); x0=1; xdot0=0;
t=np.linspace(0,100,1000);
x_IC=np.exp(-zeta*wn*t)*((xdot0/wd)*np.cos(wd*t-np.pi/2)+(x0/(np.sqrt(1-zeta**2)))*np.sin(wd*t+phi_IC))   #np.exp(-zeta*wn*t);

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=x_IC, mode='lines+markers', name='x'))
fig.update_layout(title='Initial Condition Response', xaxis=dict(title='Time / t'), yaxis=dict(title=r'$x_{IC}(t)$'))
fig.show()

**Reasoning**:
Define the physical parameters of the system and the governing equation of motion for a damped torsional pendulum.



In [ ]:
theta = dynamicsymbols('theta')
I, K, C, M, g, L, t, theta_0 = symbols('I, K, C, M, g, L, t, theta_0', real=True)

In [ ]:
theta_dot=Derivative(theta)
theta_ddot=Derivative(theta_dot)

In [ ]:
I*theta_ddot+C*theta_dot+K*theta

In [ ]:
M*g*L*cos(theta)

In [ ]:
latext(I*theta_ddot+C*theta_dot+K*theta)

## Solve the equation of motion (part a)

### Subtask:
Use Python to solve the differential equation for the given initial conditions and parameters to find the expression for the time history of the gate's motion.


**Reasoning**:
Define the initial conditions and parameters, calculate the moment of inertia, and substitute these values into the equation of motion.



In [ ]:
import numpy as np

# Define parameters
mass = 75  # kg
distance_to_cm = 1.5  # meters
k_val = 130  # N.m
c_val = 10  # N.m/s

# Calculate moment of inertia (assuming mass concentrated at a point)
I_val = mass * distance_to_cm**2

# Define initial conditions
initial_theta_deg = 30
initial_theta_rad = np.deg2rad(initial_theta_deg)  # Convert degrees to radians
initial_velocity = 0  # Released from rest

# Substitute parameter values into the equation of motion
# I * d^2(theta)/dt^2 + c * d(theta)/dt + k * theta = 0
# We'll use the numerical values directly in the solution process.
# The equation becomes: I_val * theta''(t) + c_val * theta'(t) + k_val * theta(t) = 0

**Reasoning**:
Solve the second-order linear homogeneous differential equation with the calculated coefficients and apply the initial conditions to find the particular solution for $\theta(t)$ using sympy.



In [ ]:
import sympy as sp

# Define symbolic variables
theta, t = sp.symbols('theta t', real=True)
C1, C2 = sp.symbols('C1 C2') # Constants of integration

# Define the coefficients from the previous calculations
I_val = 168.75
c_val = 10
k_val = 130

# Define the differential equation using the numerical coefficients
# I * d^2(theta)/dt^2 + c * d(theta)/dt + k * theta = 0
diffeq = sp.Eq(I_val * sp.diff(theta, t, 2) + c_val * sp.diff(theta, t) + k_val * theta, 0)

# Solve the differential equation
solution = sp.dsolve(diffeq, theta(t))

# Extract the general solution
general_solution = solution.rhs

# Define the initial conditions as equations
initial_theta_rad = np.deg2rad(30)
initial_velocity = 0

# Equation from initial displacement: theta(0) = initial_theta_rad
eq_initial_theta = sp.Eq(general_solution.subs(t, 0), initial_theta_rad)

# Equation from initial velocity: theta'(0) = initial_velocity
# First, find the derivative of the general solution
theta_prime_t = sp.diff(general_solution, t)
# Then substitute t=0 into the derivative
eq_initial_velocity = sp.Eq(theta_prime_t.subs(t, 0), initial_velocity)

# Solve for the constants of integration C1 and C2 using the initial conditions
constants = sp.solve((eq_initial_theta, eq_initial_velocity), (C1, C2))

# Substitute the values of C1 and C2 back into the general solution to get the particular solution
particular_solution = general_solution.subs(constants)

# Display the expression for theta(t)
display(sp.Eq(theta(t), particular_solution))

**Reasoning**:
The error `TypeError: 'Symbol' object is not callable` indicates that `theta(t)` is being treated as a function call on a Symbol `theta`, which is incorrect usage in `sp.dsolve`. The correct way to represent the function to be solved for is just `theta`. The code needs to be fixed by changing `theta(t)` to `theta` in the `sp.dsolve` call and ensuring that `theta` is defined as a function using `sp.Function`.



In [ ]:
import numpy as np
import sympy as sp

# Define symbolic variables, defining theta as a function of t
t = sp.symbols('t', real=True)
theta = sp.Function('theta')(t)
C1, C2 = sp.symbols('C1 C2') # Constants of integration

# Define the coefficients from the previous calculations
I_val = 168.75
c_val = 10
k_val = 130

# Define the differential equation using the numerical coefficients
# I * d^2(theta)/dt^2 + c * d(theta)/dt + k * theta = 0
diffeq = sp.Eq(I_val * sp.diff(theta, t, 2) + c_val * sp.diff(theta, t) + k_val * theta, 0)

# Solve the differential equation
solution = sp.dsolve(diffeq, theta)

# Extract the general solution
general_solution = solution.rhs

# Define the initial conditions as equations
initial_theta_deg = 30
initial_theta_rad = np.deg2rad(initial_theta_deg)  # Convert degrees to radians
initial_velocity = 0

# Equation from initial displacement: theta(0) = initial_theta_rad
eq_initial_theta = sp.Eq(general_solution.subs(t, 0), initial_theta_rad)

# Equation from initial velocity: theta'(0) = initial_velocity
# First, find the derivative of the general solution
theta_prime_t = sp.diff(general_solution, t)
# Then substitute t=0 into the derivative
eq_initial_velocity = sp.Eq(theta_prime_t.subs(t, 0), initial_velocity)

# Solve for the constants of integration C1 and C2 using the initial conditions
constants = sp.solve((eq_initial_theta, eq_initial_velocity), (C1, C2))

# Substitute the values of C1 and C2 back into the general solution to get the particular solution
particular_solution = general_solution.subs(constants)

# Display the expression for theta(t)
display(sp.Eq(theta, particular_solution))

## Explain vibration phenomenon (part b)

### Subtask:
Describe the concept of critical damping and how it relates to minimizing the closing time of the gate. Explain how the damping coefficient can be adjusted in this gate design.


**Reasoning**:
Explain the concepts of critical damping, underdamped, and overdamped systems, relating them to the gate's motion and how critical damping minimizes closing time. Then, describe how the damping coefficient can be adjusted based on the problem description.



In [ ]:
# Explanation of damping types and critical damping for minimizing closing time
print("Explanation of Damping and its effect on Gate Closing Time:\n")

print("In a damped oscillatory system like the gate, the type of damping determines how the system returns to its equilibrium position (closed gate, theta = 0). There are three main types:\n")

print("1. Underdamped: The system oscillates around the equilibrium position before settling. The damping is not strong enough to prevent overshoot. While the initial return might be fast, the subsequent oscillations prolong the time it takes to reach and stay at equilibrium.")
print("   - Behavior: Oscillatory decay.")
print("   - Gate: The gate would swing back and forth around the closed position before finally coming to rest.\n")

print("2. Overdamped: The damping is so strong that the system returns to equilibrium very slowly without any oscillation. The motion is sluggish.")
print("   - Behavior: Slow, non-oscillatory return.")
print("   - Gate: The gate would slowly creep towards the closed position without any overshoot.\n")

print("3. Critically Damped: This is the ideal case for minimizing the time to return to equilibrium without oscillation. The system returns to equilibrium as quickly as possible without overshooting.")
print("   - Behavior: Fastest possible non-oscillatory return.")
print("   - Gate: The gate would close quickly and smoothly, stopping precisely at the closed position without swinging past it.\n")

print("Relating to minimizing closing time:")
print("Critical damping is crucial for minimizing the time the gate remains open because it provides the fastest possible return to the equilibrium position (theta = 0) without any oscillations. Underdamped systems waste time oscillating, while overdamped systems take too long to reach equilibrium due to excessive resistance. Critical damping strikes the perfect balance, dissipating energy efficiently to bring the system to rest in the shortest time.\n")

print("Adjusting the damping coefficient in the gate design:")
print("According to the problem description, the torsional damping is controlled by a drum brake that uses friction. The amount of friction, and thus the damping coefficient (c), can be controlled by 'tightening or loosening a bolt which presses a brake shoe against the drum.' Tightening the bolt would increase the pressure of the brake shoe on the drum, increasing friction and thus increasing the damping coefficient. Loosening the bolt would decrease the pressure, reducing friction and the damping coefficient.")

## Calculate optimal coefficients and closing time

### Subtask:
Calculate the moment of inertia of the gate. Determine the critical damping coefficient. Calculate the time it takes for the critically damped gate to close to 1 degree from 30 degrees.


**Reasoning**:
Calculate the moment of inertia and the critical damping coefficient, then set up the symbolic variables and the differential equation for the critically damped case.



In [ ]:
import numpy as np
import sympy as sp

# Step 1: Calculate the moment of inertia (I)
mass = 75  # kg
distance_to_cm = 1.5  # meters
I_val = mass * distance_to_cm**2
print(f"Moment of inertia (I): {I_val} kg.m^2")

# Step 2: Calculate the critical damping coefficient (c_critical)
k_val = 130  # N.m
c_critical_val = 2 * np.sqrt(I_val * k_val)
print(f"Critical damping coefficient (c_critical): {c_critical_val} N.m/s")

# Step 3: Define symbolic variables and function
t = sp.symbols('t', real=True)
theta = sp.Function('theta')(t)
C1, C2 = sp.symbols('C1 C2') # Constants of integration

# Step 4: Define the differential equation for a critically damped system
# I * d^2(theta)/dt^2 + c_critical * d(theta)/dt + k * theta = 0
diffeq_critical = sp.Eq(I_val * sp.diff(theta, t, 2) + c_critical_val * sp.diff(theta, t) + k_val * theta, 0)
display("Differential equation for critical damping:", diffeq_critical)

**Reasoning**:
Solve the critically damped differential equation, extract the general solution, and define the initial conditions.



In [ ]:
# Step 5: Solve the differential equation
solution_critical = sp.dsolve(diffeq_critical, theta)
display("General solution for critical damping:", solution_critical)

# Step 7: Extract the general solution
general_solution_critical = solution_critical.rhs

# Step 8: Define the initial conditions
initial_theta_deg = 30
initial_theta_rad = np.deg2rad(initial_theta_deg)  # Convert degrees to radians
initial_velocity = 0  # Released from rest

**Reasoning**:
Use the initial conditions to set up equations for the constants of integration and solve for them. Then substitute the constants back into the general solution to get the particular solution.



In [ ]:
# Step 9: Use the initial conditions to set up equations for C1 and C2
# Equation from initial displacement: theta(0) = initial_theta_rad
eq_initial_theta_critical = sp.Eq(general_solution_critical.subs(t, 0), initial_theta_rad)

# Equation from initial velocity: theta'(0) = initial_velocity
# First, find the derivative of the general solution
theta_prime_t_critical = sp.diff(general_solution_critical, t)
# Then substitute t=0 into the derivative
eq_initial_velocity_critical = sp.Eq(theta_prime_t_critical.subs(t, 0), initial_velocity)

# Step 10: Solve for the constants of integration C1 and C2
constants_critical = sp.solve((eq_initial_theta_critical, eq_initial_velocity_critical), (C1, C2))
display("Constants of integration (C1, C2):", constants_critical)

# Step 11: Substitute the values of C1 and C2 back into the general solution to get the particular solution
particular_solution_critical = general_solution_critical.subs(constants_critical)
display("Particular solution for critical damping:", sp.Eq(theta, particular_solution_critical))

**Reasoning**:
Define the target angle for closing and solve the particular solution equation for time `t` when the angle is equal to the target angle, selecting the smallest positive real solution.



In [ ]:
# Step 12: Define the target angle for closing (1 degree converted to radians)
target_theta_deg = 1
target_theta_rad = np.deg2rad(target_theta_deg)

# Step 13: Set the particular solution equal to the target angle and solve for t
# Use numerical evaluation for solving since the expression is complex
# We need to find t such that particular_solution_critical - target_theta_rad = 0

# Define a numerical function from the symbolic expression
theta_critical_func = sp.lambdify(t, particular_solution_critical - target_theta_rad, 'numpy')

# Find the root(s) of the function. Since we expect a single root for critical damping
# as it approaches the target without oscillation, we can use a root-finding algorithm.
# We need to provide an initial guess or a bracket. Let's try a few guesses or a range.
# We know the gate starts at 30 deg and closes towards 0 deg, so the time should be positive.
# A reasonable initial guess might be around the time constant or a few seconds.

# Let's try to find the root numerically. We can use scipy.optimize.fsolve or root.
# Since we are in a Jupyter environment, we can assume scipy is available or install it if needed.
# Let's assume scipy is available for now.

from scipy.optimize import fsolve

# Initial guess for t (can be refined based on expected time scale)
# The time constant for critical damping is tau = 2*I/c_critical = 2*168.75 / 296.226 = 1.14 seconds approx.
# Let's guess a time slightly larger than this.
initial_guess = 2.0

# Solve for t numerically
time_to_close_critical_solutions = fsolve(theta_critical_func, initial_guess)

# fsolve returns an array of solutions; we expect one real positive solution.
# Check if the solution is real and positive
time_to_close_critical = None
if time_to_close_critical_solutions.size > 0:
    # Filter for positive real solutions
    positive_real_solutions = time_to_close_critical_solutions[(time_to_close_critical_solutions.imag == 0) & (time_to_close_critical_solutions > 0)]
    if positive_real_solutions.size > 0:
        # Choose the smallest positive real solution
        time_to_close_critical = np.min(positive_real_solutions.real)

if time_to_close_critical is not None:
    print(f"Time to close to {target_theta_deg} degrees (critically damped): {time_to_close_critical:.4f} seconds")
else:
    print(f"Could not find a positive real time to close to {target_theta_deg} degrees with critical damping.")

## Generate pdf report (part d)

### Subtask:
Create a PDF document containing the working, the final expression for the motion, the explanation of critical damping, the calculated optimal coefficients and closing time, and the calculations.


## Summary:

### Data Analysis Key Findings

*   The system was identified as a damped torsional pendulum governed by the equation: $I \frac{d^2\theta}{dt^2} + c \frac{d\theta}{dt} + k \theta = 0$.
*   For the initial damping coefficient ($c=10$ N.m/s), the particular solution for the gate's motion $\theta(t)$ was derived based on initial conditions of 30 degrees angular displacement and zero initial velocity.
*   Critical damping is defined as the damping level that allows the system to return to equilibrium as quickly as possible without oscillation, which is optimal for minimizing the gate's closing time.
*   The gate's damping coefficient can be adjusted physically by tightening or loosening a bolt controlling a brake shoe against a drum, thereby changing the friction.
*   The moment of inertia of the gate was calculated to be $168.75 \, \text{kg.m}^2$, based on a mass of 75 kg and a center of mass at 1.5m from the pivot.
*   The critical damping coefficient for this gate system was calculated to be approximately $296.226 \, \text{N.m/s}$. This is significantly higher than the initial damping of 10 N.m/s, indicating the initial configuration is heavily underdamped.
*   Under critical damping, the time taken for the gate to close from an initial angle of 30 degrees to within 1 degree of the closed position (0 degrees) is approximately $5.9595$ seconds.

### Insights or Next Steps

*   Adjusting the damping coefficient to the critical damping value (approx. 296.23 N.m/s) is crucial to optimize the gate's closing performance, ensuring it closes quickly and smoothly without unwanted oscillations.
*   Further analysis could investigate the sensitivity of the closing time to small deviations from the critical damping coefficient or explore alternative damping mechanisms if the current brake design is not capable of reaching the optimal damping level.
